# Week 07 – Penetration Testing  
## Networks and Systems Security

This notebook implements the **Week 07 lab** on Penetration Testing using Python.

It focuses on the early stages of a penetration test:

- Passive and active **reconnaissance**
- **Basic scanning** and port enumeration
- Using Python-only tools and an optional `python-nmap` wrapper


## Aims of the Seminar

This workshop is designed to provide a **hands-on introduction** to some core concepts of **Penetration Testing (Pen Testing)**.

We will:

- Use Python to simulate and understand the **initial stages** of a penetration test.  
- Focus on **reconnaissance** and **vulnerability assessment**.  
- Show how scriptable tooling supports **ethical hacking** and security testing.

> 🔐 Penetration testing is **not** a magic bullet. It must **supplement** a comprehensive security strategy (secure design, hardening, monitoring, incident response, etc.).


## Learning Objectives

By the end of this lab, you should be able to:

- Understand the **purpose and principles** of penetration testing.  
- Explain key phases: **reconnaissance, scanning, enumeration, vulnerability assessment, exploitation, reporting**.  
- Use Python to perform **basic passive and active recon** against authorised targets.  
- Simulate **black-box** style testing using HTTP headers and simple port scans.  
- Integrate or extend with external tools like **Nmap** via `python-nmap`.


## Ethics & Legality ⚖️

**You must follow these rules at all times:**

- ✅ Only test systems you **own** or that you have **explicit, written permission** to test.  
- ✅ Use a **designated, isolated test environment**, e.g.:  
  - Local virtual machines  
  - A dedicated lab network  
  - Training platforms such as **Hack The Box**, **TryHackMe**, **OverTheWire**, etc.  
- ❌ **Never** scan random public systems or networks without permission.  
- ❌ Unauthorised testing is **illegal** and **unethical**.

The scripts in this notebook are for **educational use only**, within your own controlled environment.


## Passive Reconnaissance – Domain & IP Information

A **Whois-style lookup** or IP intelligence query allows you to discover:

- Approximate **organisation / ISP**  
- **City / country** associated with an IP  
- Sometimes hints about infrastructure and hosting

Here, we demonstrate a basic, safe example using a public IP information API and Python.


In [1]:
import socket
import requests

def get_domain_info(domain: str):
    """Resolve a domain to an IP and query basic public IP information."""
    try:
        ip = socket.gethostbyname(domain)
        print(f"[+] Domain: {domain}")
        print(f"[+] IP Address: {ip}")

        resp = requests.get(f"https://ipapi.co/{ip}/json/", timeout=5)
        if resp.status_code == 200:
            data = resp.json()
            print(f"    Organization: {data.get('org', 'Unknown')}")
            print(f"    City:         {data.get('city', 'Unknown')}")
            print(f"    Country:      {data.get('country_name', 'Unknown')}")
        else:
            print("[!] Could not fetch IP information (non-200 status).")
    except Exception as e:
        print(f"[!] Error: {e}")

# Example: use a well-known public domain (read-only, no intrusive scanning)
get_domain_info("python.org")


[+] Domain: python.org
[+] IP Address: 151.101.192.223
[!] Could not fetch IP information (non-200 status).


## Practical – Simple Black-Box Web Recon (HTTP HEAD)

We can simulate a tiny part of a **black box** test by making a `HEAD` request to a URL and inspecting HTTP response headers.

This can reveal:

- Web server type (if not hidden)  
- Basic content type  
- Some security-relevant headers (not shown here)


In [2]:
import requests

def black_box_recon(url: str):
    """Send a HEAD request and print basic headers (black-box style)."""
    try:
        resp = requests.head(url, timeout=5)
        print(f"[+] URL: {url}")
        print("Black Box Findings:")
        print(f"  Server:       {resp.headers.get('Server', 'Unknown')}")
        print(f"  Content-Type: {resp.headers.get('Content-Type', 'Unknown')}")
    except Exception as e:
        print(f"[!] Error: {e}")

# Example: public site (read-only HTTP metadata)
test_url = "https://www.python.org"
black_box_recon(test_url)


[+] URL: https://www.python.org
Black Box Findings:
  Server:       Unknown
  Content-Type: text/html; charset=utf-8


## Practical – Basic Port Scanner (Python Only)

We can simulate part of the **scanning/enumeration** phase using pure Python.




In [3]:
import socket

def scan_ports(host: str, ports):
    """Very simple TCP connect scan against a list of ports."""
    open_ports = []
    for port in ports:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex((host, port))
        if result == 0:
            open_ports.append(port)
        sock.close()
    return open_ports

host = "127.0.0.1"  # localhost only
ports_to_test = [22, 80, 443, 8080]

print(f"[+] Scanning {host} on ports {ports_to_test}...")
open_ports = scan_ports(host, ports_to_test)
print(f"[+] Open ports on {host}: {open_ports}")


[+] Scanning 127.0.0.1 on ports [22, 80, 443, 8080]...
[+] Open ports on 127.0.0.1: []


## Using Nmap via `python-nmap` 
Here we show how to call Nmap from Python using the `python-nmap` package.


In [5]:

%pip install python-nmap

import nmap

def nmap_scan(host: str, port_range: str = "1-1024"):
    """Wrapper around nmap.PortScanner() for a simple scan."""
    nm = nmap.PortScanner()
    try:
        print(f"[+] Running Nmap scan against {host} (ports {port_range})...")
        nm.scan(hosts=host, ports=port_range, arguments='-sV')  # -sV for service version detection

        for target in nm.all_hosts():
            print(f"\nHost: {target} ({nm[target].hostname()})")
            print(f"State: {nm[target].state()}")
            for proto in nm[target].all_protocols():
                print(f"Protocol: {proto}")
                lport = nm[target][proto].keys()
                for port in sorted(lport):
                    service = nm[target][proto][port]
                    state = service.get('state', 'unknown')
                    name = service.get('name', 'unknown')
                    version = service.get('version', '')
                    print(f"  Port: {port}\tState: {state}\tService: {name} {version}")
    except Exception as e:
        print(f"[!] Error while running Nmap scan: {e}")

# Example: Scanning localhost for a small port range (uncomment to run)
# nmap_scan("127.0.0.1", "1-100")


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for python-nmap: filename=python_nmap-0.7.1-py2.py3-none-any.whl size=20679 sha256=480018761e2e97543620135f23dc5fbf278047a3ca96f92f9c33eded97328231
  Stored in directory: /Users/dissept/Library/Caches/pip/wheels/bd/bb/2d/06d2d27ec0b19037b0534b212d830cdcc43ddbe38755e7e129
Successfully built python-nmap
Note: you may need to restart the kernel to use updated packages.


## Reflection

After working through this notebook, reflect on the following:

1. Passive Recon Findings

From the passive recon step, I was able to gather the domain’s IP address, the organisation that owns the IP, and the approximate location (city/country). This gives a basic overview of who operates the infrastructure without directly interacting with the target, which is useful when starting a security assessment.

2. Black-Box HTTP Header Results

The black-box HTTP check revealed general server information, such as the server type (if not hidden) and the content type being served. If the server header was missing or anonymised, it suggests the site has been hardened to reduce fingerprinting.

3. Local Port Scan Results

The port scanner identified which common ports were open on my localhost. Depending on the machine, this usually includes ports like 22 (SSH), 80/443 (web services), or 8080 (development servers). The simple scanner can’t confirm what is running, but the ports give a good indication of likely services.

4. Difference Between Simple Scanner and python-nmap

python-nmap produced far more detailed results than the basic scanner. It showed service names, version information, and protocol details, while the simple Python scanner only reported whether a port was open. Nmap essentially performs real enumeration, whereas the basic script is just a connectivity test.

5. How I Would Extend the Scripts

Scan more hosts: Loop through an IP range or use Python’s ipaddress module.
Reporting: Save results to JSON/CSV and auto-generate a summary.
Vulnerability checking: Compare detected versions with known CVEs or use Nmap’s NSE scripts.

6. Importance of Permission & Scope

Penetration testing without permission is illegal and can disrupt systems. Having a clearly defined scope protects both the tester and the organisation, ensures the test is safe, and prevents accidental scanning of third-party systems.

